In [1]:
import pandas as pd
from sqlalchemy import create_engine
import yaml
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
# Read csv
trade_df = pd.read_csv("./data/trade.csv", encoding="utf-8", usecols=['BYRORGNM1', 'SPLYORGNM1'], skipinitialspace=True,
                       dtype={'HSCD': str, "QTY": float, 'BYRADDR2': str, 'SPLYADDR2': str})
trade_df.head()

,BYRORGNM1,SPLYORGNM1
0,주식회사 포스코대우,POSCO
1,주식회사 포스코대우,POSCO
2,주식회사 포스코대우,POSCO
3,주식회사 포스코대우,POSCO
4,주식회사 포스코대우,POSCO


In [3]:
# Merge name
csv_company_series = trade_df['BYRORGNM1'].append(trade_df['SPLYORGNM1'], ignore_index=True).drop_duplicates().reset_index(drop=True)
csv_company_series.head()

0         주식회사 포스코대우
1        도레이케미칼 주식회사
2    도레이첨단소재주식회사 4공장
3        도레이첨단소재주식회사
4    도레이첨단소재주식회사 2공장
dtype: object

In [4]:
len(csv_company_series)

53300

In [5]:
stopword_test = csv_company_series.str.replace("주식회사", "").str.replace("\(주\)", "")\
        .str.replace("\(유\)", "").str.replace("㈜", "").str.replace(" ", "")
stopword_test.head()

0         포스코대우
1        도레이케미칼
2    도레이첨단소재4공장
3       도레이첨단소재
4    도레이첨단소재2공장
dtype: object

In [8]:
# make tfidf matrix set
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2,3))
tfidf_matrix = tfidf.fit_transform(stopword_test)

In [9]:
tfidf_matrix.shape

(53300, 92828)

In [10]:
tfidf.vocabulary_

{'알킨스': 51462,
 '티시스': 83195,
 '봉화엔': 35352,
 '미코아': 32473,
 '드바이': 22564,
 '슨(': 46725,
 '.f': 1785,
 '트윈': 82724,
 '서울': 40207,
 '전선공': 68816,
 '주진': 71453,
 '음료청': 63173,
 '모코리': 31299,
 '금당': 13401,
 '로링크': 25985,
 '연나': 56045,
 'esh': 3924,
 '태일실': 80420,
 '현홀딩': 90266,
 '티엔푸': 83313,
 '펫트': 84803,
 '피온': 86794,
 '성잉크': 41887,
 '일전기': 67519,
 '상상': 39425,
 '츠무': 75743,
 '타티': 79654,
 '더페': 19848,
 '화트림': 91624,
 '아산-': 50168,
 'lho': 5722,
 '가네마': 8756,
 '츠(': 75724,
 '탈피': 79842,
 ')상사': 1101,
 '아이아': 50505,
 '천다': 74823,
 '루컴즈': 27064,
 '터파': 80863,
 '우진정': 59621,
 '씨(g': 49106,
 '빅스톤': 37181,
 '월씰': 61292,
 '텍스여': 81344,
 '대한잉': 19406,
 '콥스전': 78339,
 '준글': 71617,
 '날푸': 15026,
 '즈유': 72062,
 '로와이': 26274,
 '신폴리': 48343,
 '105': 2161,
 '파낙스': 83758,
 '린소': 28640,
 '네보': 15639,
 '더원': 19779,
 '블레스': 36266,
 '임하우': 67922,
 '백엘이': 33644,
 '에쓰전': 54064,
 '주피엘': 71557,
 '산에프': 38184,
 '호조': 90780,
 '어필름': 53367,
 '린노알': 28592,
 '금강솔': 13357,
 '후드코': 92069,
 '어업': 53271,
 '호타이': 9082

In [11]:
# Set company for comparing
company_i = 0
cosine_sim = linear_kernel(tfidf_matrix[company_i], tfidf_matrix)
print(csv_company_series[company_i])

주식회사 포스코대우


In [14]:
# Result comparing
sorted_index = sorted(range(len(cosine_sim[0])), key=lambda i: cosine_sim[0][i])[-30:]
sorted_index.reverse()
for i in sorted_index:
    print(str(i), csv_company_series[i], str(cosine_sim[0][i]))

23510 주식회사포스코대우 1.0
21305 포스코 대우 1.0
21020 (주)포스코대우 1.0
18574 포스코대우 1.0
0 주식회사 포스코대우 1.0
50270 주식회사 포스코대우T 0.8396449333639798
51863 주식회사포스코 0.5883237370877368
22042 (주)포스코 0.5883237370877368
19428 주식회사 포스코 0.5883237370877368
49717 주식회사 포스코대우 STS사업부 0.5675971756265995
20173 포스코대우 TMC사업부 0.557354894125328
38793 주식회사포스코대우STS 사업부 안산 0.5074446681797886
34492 주식회사 포스코대우 STS 사업부 안산 0.5074446681797886
21529 주식회사 포스코대우 STS사업부 안산 0.5074446681797886
20758 (주)포스코대우 STS사업부 안산 0.5074446681797886
17085 주식회사 포스코대우TMC 사업부 천안 0.4956238437481716
43215 (주) 포스코대우 STS사업부 안산사업 0.45146490029844566
30098 (주)포스코대우 STS 사업부 안산사업 0.45146490029844566
53029 (주)포스코대우 STS사업부 부산사업장 0.4327141556997126
43377 (주)포스코대우TMC사업부 포항사업장 0.42571366529612353
31967 (주)포스코대우 TMC사업부 포항사업장 0.42571366529612353
17483 ㈜포스코대우 TMC사업부 포항사업장 0.42571366529612353
18459 ㈜포스코대우TMC사업부 천안사업장 0.425275886316196
18575 주포스코 0.3670538604291159
283 (주)포스코건설 0.3159926135131049
51721 포스코켐텍 0.3154629890037671
11654 (주)포스코켐텍 0.3154629890037671
23539 (주)포스코엠

In [75]:
test_name = ['포스코', '도레이케미칼', '포스코대우', '(주)우익반도체']
test_matrix = tfidf.transform(test_name)
test_matrix.shape

(4, 92828)

In [76]:
company_i = 0
cosine_sim = linear_kernel(test_matrix[company_i], tfidf_matrix)

주식회사 포스코대우


In [77]:
sorted_index = sorted(range(len(cosine_sim[0])), key=lambda i: cosine_sim[0][i])[-100:]
sorted_index.reverse()

In [78]:
for i in sorted_index:
    print(csv_company_series[i], str(cosine_sim[0][i]))

주식회사포스코 1.0
(주)포스코 1.0
주식회사 포스코 1.0
주포스코 0.6238977577992526
주식회사포스코대우 0.5883237370877369
포스코 대우 0.5883237370877369
(주)포스코대우 0.5883237370877369
포스코대우 0.5883237370877369
주식회사 포스코대우 0.5883237370877369
(주)포스코건설 0.537106687344116
포스코켐텍 0.536206461030013
(주)포스코켐텍 0.536206461030013
(주)포스코엠텍 0.5286166856736463
주식회사 포스코대우T 0.4939830450234805
포스코강판주식회사 0.4881832918746077
포스코ICT 0.4580983712596689
(주)포스코플랜텍 0.45402337903203593
(주)포스코ESM 0.44751042517186407
주식회사포스코(포항) 0.38657047692157703
(주)포스코건설-본사 0.36255959165907936
주식회사 포스코대우 STS사업부 0.33393089152508554
포스코대우 TMC사업부 0.3279051141959529
(주)포스텍 0.3157743004635862
주식회사 포스텍 0.3157743004635862
주식회사포스코대우STS 사업부 안산 0.2985417435487799
주식회사 포스코대우 STS 사업부 안산 0.2985417435487799
주식회사 포스코대우 STS사업부 안산 0.2985417435487799
(주)포스코대우 STS사업부 안산 0.2985417435487799
(주)포스엔 0.29363378318560013
주식회사 포스코대우TMC 사업부 천안 0.2915872719437129
(주)포스프 0.2911344418307872
주식회사 포스포 0.28796483725558736
(주)포스포 0.28796483725558736
(주)포스켐 0.28461818798943445
(주)포스콤 0.27444290306319064
(

In [44]:
pd.concat([csv_company_series, csv_company_series], axis=1, keys=['before', 'after'])

,before,after
0,주식회사 포스코대우,주식회사 포스코대우
1,도레이케미칼 주식회사,도레이케미칼 주식회사
2,도레이첨단소재주식회사 4공장,도레이첨단소재주식회사 4공장
3,도레이첨단소재주식회사,도레이첨단소재주식회사
4,도레이첨단소재주식회사 2공장,도레이첨단소재주식회사 2공장
5,도레이첨단소재주식회사 군산공장,도레이첨단소재주식회사 군산공장
6,티씨케이텍스타일 주식회사,티씨케이텍스타일 주식회사
7,티씨케이텍스타일(주),티씨케이텍스타일(주)
8,티씨케이텍스타일주식회사,티씨케이텍스타일주식회사
9,주식회사 에스엘라이팅,주식회사 에스엘라이팅


In [5]:
config = yaml.safe_load(open('dart_config.yml'))
db_url = 'mysql://' + config['DB_USER'] + ':' + config['DB_USER_PASSWORD'] + '@' + config['DB_HOST'] \
                            + '/' + config['DB_NAME'] + '?charset=utf8'
engine = create_engine(db_url, encoding='utf-8')
conn = engine.connect()

In [6]:
db_company_df = pd.read_sql_table(config['DB_TABLE_NAME_1'], con=conn, index_col='id', 
                          columns=['id', 'name', '업종', '상장일', '지역'])
db_company_df.head()

,name,업종,상장일,지역
id,,,,
1,롯데케미칼,기초 화학물질 제조업,1991-05-30,서울특별시
2,대한유화,기초 화학물질 제조업,1999-08-11,서울특별시
3,이화산업,기타 전문 도매업,1994-04-29,서울특별시
4,디케이앤디,플라스틱제품 제조업,2018-11-20,경기도
5,송원산업,기타 화학제품 제조업,1977-06-25,울산광역시


In [8]:
db_company_list = db_company_df['name'].drop_duplicates().reset_index(drop=True)
db_company_list.head()

0    롯데케미칼
1     대한유화
2     이화산업
3    디케이앤디
4     송원산업
Name: name, dtype: object

In [85]:
cosine_sim = linear_kernel(tfidf_matrix[1], tfidf_matrix)

In [86]:
cosine_sim[0]

array([0.04649235, 1.        , 0.1422452 , ..., 0.        , 0.        ,
       0.        ])

In [87]:
sorted(range(len(cosine_sim[0])), key=lambda i: cosine_sim[0][i])[-5:]

[35, 3, 220, 18, 1]

In [99]:
csv_company_list[1]

'도레이케미칼 주식회사'

In [57]:
len(csv_company_list)

796